## BME 544 Assignment 6

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import iirnotch, freqz, sosfiltfilt, hilbert, butter, filtfilt, welch

In [ ]:
filename = "DC_4/MVC.txt"

# Load data (skip header)
data = np.loadtxt(filename, skiprows=1)

# Separate columns
time = data[:, 0]
sig = data[:, 1]

# Sampling rate and FFT
fs = 1000  # Hz
n = len(sig)

### 2. Maximum Voluntary Contraction (MVC)

In [ ]:
interval = [1.2, 12.5]

analytic_signal = hilbert(sig)
envelope = np.abs(analytic_signal)

cutoff = 5
b, a = butter(N=4, Wn=cutoff / (0.5 * fs), btype='low')
envelope_filtered = filtfilt(b, a, envelope)

mask = (time >= interval[0]) & (time <= interval[1])
avg_env = np.mean(envelope_filtered[mask])

envelope_normalized = envelope_filtered / avg_env

plt.figure(figsize=(12, 6))
plt.plot(time, envelope_normalized, label='Normalized Envelope', linewidth=2)
plt.xlabel('Time [s]')
plt.ylabel('EMG Amplitude (Normalized)')
plt.title(f'Normalized Envelope of Signal (MVC envelope segment average = {avg_env:.2f})')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

### 3. EMG for two levels of force

In [ ]:
filename = "DC_4/Different Objects.txt"
intervals = [(3.8, 10.8), (16, 24.5)]

data = np.loadtxt(filename, skiprows=1)

time = data[:, 0]
sig = data[:, 1]

fs = 1000
n = len(sig)

analytic_signal = hilbert(sig)
envelope = np.abs(analytic_signal)

cutoff = 5
b, a = butter(N=4, Wn=cutoff / (0.5 * fs), btype='low')
envelope_filtered = filtfilt(b, a, envelope)

envelope_normalized = envelope_filtered / avg_env

plt.figure(figsize=(12, 6))
plt.plot(time, envelope_normalized, label='Normalized Envelope', linewidth=2)
plt.xlabel('Time [s]')
plt.ylabel('EMG Amplitude (Normalized)')
plt.title(f'Normalized Envelope of Different Objects.txt')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
for i, (start, end) in enumerate(intervals):
    mask = (time >= start) & (time <= end)
    segment = sig[mask]

    f, Pxx = welch(segment, fs=fs, nperseg=1024)

    mean_freq = np.sum(f * Pxx) / np.sum(Pxx)

    cumulative_power = np.cumsum(Pxx)
    total_power = cumulative_power[-1]
    median_freq = f[np.searchsorted(cumulative_power, total_power / 2)]

    plt.semilogy(f, Pxx, label=f'Interval {i+1}: {start}-{end}s')

    print(f"Interval {i+1} ({start}-{end}s):")
    print(f"  Mean Frequency: {mean_freq:.2f} Hz")
    print(f"  Median Frequency: {median_freq:.2f} Hz\n")

plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD [V²/Hz]')
plt.title('Different Objects PSD')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### 4. Sustained (sub-MVC) force

In [ ]:
filename = "DC_4/Objects Fatigue.txt"
intervals = [[3, 8], [59, 64]]

data = np.loadtxt(filename, skiprows=1)

time = data[:, 0]
sig = data[:, 1]

fs = 1000
n = len(sig)

analytic_signal = hilbert(sig)
envelope = np.abs(analytic_signal)

cutoff = 5
b, a = butter(N=4, Wn=cutoff / (0.5 * fs), btype='low')
envelope_filtered = filtfilt(b, a, envelope)

envelope_normalized = envelope_filtered / avg_env

plt.figure(figsize=(12, 6))
for i, interval in enumerate(intervals):
    plt.axvline(x=interval[0], color='red', linestyle='--', label=f'Interval {i+1} Start: {interval[0]}s')
    plt.axvline(x=interval[1], color='blue', linestyle='--', label=f'Interval {i+1} End: {interval[1]}s')
plt.plot(time, envelope_normalized, label='Normalized Envelope', linewidth=2)
plt.xlabel('Time [s]')
plt.ylabel('EMG Amplitude (Normalized)')
plt.title(f'Normalized Envelope of Objects Fatigue.txt')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
for i, (start, end) in enumerate(intervals):
    mask = (time >= start) & (time <= end)
    segment = sig[mask]

    f, Pxx = welch(segment, fs=fs, nperseg=1024)

    mean_freq = np.sum(f * Pxx) / np.sum(Pxx)

    cumulative_power = np.cumsum(Pxx)
    total_power = cumulative_power[-1]
    median_freq = f[np.searchsorted(cumulative_power, total_power / 2)]

    plt.semilogy(f, Pxx, label=f'Interval {i+1}: {start}-{end}s')

    print(f"Interval {i+1} ({start}-{end}s):")
    print(f"  Mean Frequency: {mean_freq:.2f} Hz")
    print(f"  Median Frequency: {median_freq:.2f} Hz\n")

plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD [V²/Hz]')
plt.title('Objects Fatigue.txt PSD')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### 5. Sustained MVC

In [ ]:
filename = "DC_4/MVC Fatigue.txt"
intervals = [(3, 8), (12, 17)]

data = np.loadtxt(filename, skiprows=1)

time = data[:, 0]
sig = data[:, 1]

fs = 1000
n = len(sig)

analytic_signal = hilbert(sig)
envelope = np.abs(analytic_signal)

cutoff = 5
b, a = butter(N=4, Wn=cutoff / (0.5 * fs), btype='low')
envelope_filtered = filtfilt(b, a, envelope)

envelope_normalized = envelope_filtered / avg_env

plt.figure(figsize=(12, 6))
for i, interval in enumerate(intervals):
    plt.axvline(x=interval[0], color='red', linestyle='--', label=f'Interval {i+1} Start: {interval[0]}s')
    plt.axvline(x=interval[1], color='blue', linestyle='--', label=f'Interval {i+1} End: {interval[1]}s')
plt.plot(time, envelope_normalized, label='Normalized Envelope', linewidth=2)
plt.xlabel('Time [s]')
plt.ylabel('EMG Amplitude (Normalized)')
plt.title(f'Normalized Envelope of Objects Fatigue.txt')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
for i, (start, end) in enumerate(intervals):
    mask = (time >= start) & (time <= end)
    segment = sig[mask]

    f, Pxx = welch(segment, fs=fs, nperseg=1024)

    mean_freq = np.sum(f * Pxx) / np.sum(Pxx)

    cumulative_power = np.cumsum(Pxx)
    total_power = cumulative_power[-1]
    median_freq = f[np.searchsorted(cumulative_power, total_power / 2)]

    plt.semilogy(f, Pxx, label=f'Interval {i+1}: {start}-{end}s')

    print(f"Interval {i+1} ({start}-{end}s):")
    print(f"  Mean Frequency: {mean_freq:.2f} Hz")
    print(f"  Median Frequency: {median_freq:.2f} Hz\n")

plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD [V²/Hz]')
plt.title('Objects Fatigue.txt PSD')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()